<a href="https://colab.research.google.com/github/LiadTssf/cloud-project/blob/main/hw2/part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install firebase-admin
!pip install firebase

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import requests
from bs4 import BeautifulSoup
import re
from nltk.stem import PorterStemmer
from collections import Counter
import firebase_admin
from firebase_admin import credentials, db
import matplotlib.pyplot as plt
from firebase import firebase
import json
import pandas as pd


def fetch_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        return None

def index_words(soup):
    index = {}
    words = re.findall(r'\w+', soup.get_text())
    for word in words:
        word = word.lower()
        if word in index:
            index[word] += 1
        else:
            index[word] = 1
    return index

def remove_stop_words(index):
    stop_words = {'a', 'an', 'the', 'and', 'or', 'in', 'on', 'at', 'to','also','of','n'}
    for stop_word in stop_words:
        if stop_word in index:
            del index[stop_word]
    return index

def apply_stemming(index):
    stemmer = PorterStemmer()
    stemmed_index = {}
    for word, count in index.items():
        stemmed_word = stemmer.stem(word)
        if stemmed_word in stemmed_index:
            stemmed_index[stemmed_word] += count
        else:
            stemmed_index[stemmed_word] = count

    return stemmed_index
"""
def search(query, index):

    stemmer = PorterStemmer()
    query_words = re.findall(r'\w+', query.lower())
    print(query)
    results = {}
    for word in query_words:
        word = stemmer.stem(word)
        if word in index:
            results[word] = index[word]
    return results
"""
def search_engine(url):
    soup = fetch_page(url)
    if soup is None:
        return None
    index = index_words(soup)
    index = remove_stop_words(index)
    index = apply_stemming(index)
    #results = search(query, index)
    return index
'''
# Fetch and process the Onshape glossary
url = 'https://cad.onshape.com/help/Content/Glossary/glossary.htm?tocpath=_____19'
index = search_engine(url)

# Select the top 10 most significant words
most_common_words = Counter(index).most_common(10)
print("most_common_words" ,most_common_words)
significant_counts = dict(most_common_words)


print("transformed_data",transformed_data)
'''

# Initialize Firebase using your method
FBconn = firebase.FirebaseApplication('https://test100000-ccdb4-default-rtdb.asia-southeast1.firebasedatabase.app/', None)

# Store the word counts in Firebase
#result = FBconn.put('/glossary', 'words', transformed_data)
#print("Data stored successfully in Firebase:", result)

# Select the top 10 most significant words
most_common_words = Counter(index).most_common(10)
print("most_common_words" ,most_common_words)
significant_counts = dict(most_common_words)


def save_glossary_index_to_firebase():
    url = 'https://cad.onshape.com/help/Content/Glossary/glossary.htm?tocpath=_____19'
    index = search_engine(url)
    significant_words = [{'term': word, 'freq': freq} for word, freq in index.items()]
    # Transform the dictionary into a list of dictionaries with term and freq
    transformed_data = [{'term': term, 'freq': freq} for term, freq in significant_counts.items()]
    # Retrieve the data
    FBconn.put('/glossary', 'index', transformed_data)
    print("Glossary index stored successfully in Firebase")

# Save JSON data to Firebase
def save_json_to_firebase(file_path):
    with open(file_path, 'r') as file:
        team_data = json.load(file)
    FBconn.put('/team_data2', 'data', team_data)
    print("Team data stored successfully in Firebase")


#file_path = '/content/drive/MyDrive/Colab Notebooks/teamA.json_label.json'
file_path = '/content/drive/MyDrive/Colab Notebooks/Audit Trail Dashboard (1).json'

'''
# Convert to DataFrame
df = pd.DataFrame(data)

# Convert 'Time' to datetime
df['Time'] = pd.to_datetime(df['Time'])
'''

save_glossary_index_to_firebase()
save_json_to_firebase(file_path)

# Data from Firebase
#words = [item['term'] for item in data]
#counts = [item['freq'] for item in data]

# Save glossary index to Firebase

"""
# Plot the data
plt.figure(figsize=(10, 6))
plt.bar(words, counts, color='skyblue')
plt.xlabel('Words')
plt.ylabel('Counts')
plt.title('Word Frequency in Glossary Page')
plt.xticks(rotation=45)
plt.show()
"""
'''
# Plot the data
plt.figure(figsize=(10, 6))
plt.bar(words, counts, color='skyblue')
plt.xlabel('Words')
plt.ylabel('Counts')
plt.title('Word Frequency in Glossary Page')
plt.xticks(rotation=45)
plt.show()
'''

most_common_words [('context', 676), ('type', 656), ('all', 644), ('see', 638), ('keyboard', 631), ('shortcut', 631), ('plan', 626), ('part', 524), ('studio', 369), ('assembl', 279)]
Glossary index stored successfully in Firebase
Team data stored successfully in Firebase


"\n# Plot the data\nplt.figure(figsize=(10, 6))\nplt.bar(words, counts, color='skyblue')\nplt.xlabel('Words')\nplt.ylabel('Counts')\nplt.title('Word Frequency in Glossary Page')\nplt.xticks(rotation=45)\nplt.show()\n"